In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
fpath = '/content/drive/MyDrive/ColabNotebooks/ie4213/latest_hdb.csv'
Model = pd.read_csv(fpath, index_col = 0)
Model.info()

In [ ]:
Model.head()

In [ ]:
Model_copy = Model.copy()

### Month

In [ ]:
month_list = Model['month']
np.unique(month_list)

In [ ]:
# plt.figure(figsize=(45,8))
# sns.set_theme(style="whitegrid")
# sns.swarmplot(x='month',y='resale_price', data=Model_copy, s=1)

### Remaining Lease (Not Applicable) / Lease Commence Date

In [ ]:
# def lease_convert(string):
#     string_list = string.split()
#     if len(string_list) == 2:
#         return float(string_list[0])
#     else:
#         year = float(string_list[0])
#         month = float(string_list[2])/12
#         return year + month

In [ ]:
# rem_lease = Model['remaining_lease']
# rem_lease_val = []
# for e in rem_lease:
#     rem_lease_val.append(lease_convert(e))

In [ ]:
# Model_copy['remaining_lease_val'] = rem_lease_val

In [ ]:
# plt.scatter(Model_copy['remaining_lease_val'],Model_copy['resale_price'])
# plt.xlabel('remaining lease_val')
# plt.ylabel('resale_price')
# plt.show()

In [ ]:
plt.scatter(Model_copy['lease_commence_date'],Model_copy['resale_price'])
plt.xlabel('lease_commence_date')
plt.ylabel('resale_price')
plt.show()

### Storey Range

In [ ]:
Model['storey_med'].unique()

In [ ]:
plt.scatter(Model_copy['storey_med'],Model_copy['resale_price'])
plt.xlabel('storey_med')
plt.ylabel('resale_price')
plt.show()

### Flat Type

In [ ]:
Model['flat_type'].unique()

In [ ]:
# sns.swarmplot(x='storey_range_ave_val',y='resale_price', data=Model_copy, s=1)

### Floor Area Sqm

In [ ]:
plt.scatter(Model_copy['floor_area_sqm'],Model_copy['resale_price'])
plt.xlabel('floor_area_sqm')
plt.ylabel('resale_price')
plt.show()

### Others

In [ ]:
Model_copy['resale_price']

In [ ]:
Model.hist(bins=50, figsize=(10, 8))
plt.show()

In [ ]:
Model_copy.head()

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

Model['12month'] = (Model['month'].str.match(".*-12-.*"))
Model_train = Model[Model['12month'] == False]
Model_test = Model[Model['12month'] == True]

pipeline = ColumnTransformer([
("o", OrdinalEncoder(), ["month","flat_type"]),
("n", OneHotEncoder(), ["town","flat_model","market","region"]),
], remainder='passthrough')
X_train = pipeline.fit_transform(Model_train.drop(["resale_price","12month"], axis=1))
y_train = Model_train["resale_price"]
X_test = pipeline.fit_transform(Model_test.drop(["resale_price","12month"], axis=1))
y_test = Model_test["resale_price"]

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)
y_test_pred = lr.predict(X_test)
# print("Intercept: ", lr.intercept_)
# print("Coefficient: ", lr.coef_)
# print("R-sq of model: ", lr.score(X, y))
print("RMSE: %.2f" % mean_squared_error(y_test,y_test_pred, squared=False))
print("R-sq of predictions: ", r2_score(y_test,y_test_pred))

RMSE: 174923.99
R-sq of predictions:  -0.233590966301487


In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly_train = poly.fit_transform(X_train)
X_poly_test = poly.fit_transform(X_test)
lr.fit(X_poly_train, y_train)
# print("Intercept: ", lr.intercept_)
# print("Coefficient: ", lr.coef_)
# print("R-sq of model: ", lr.score(X_poly, y))
y_test_pred = lr.predict(X_poly_test)
print("RMSE: %.2f" % mean_squared_error(y_test,y_test_pred, squared=False))
print("R-sq of predictions: ", r2_score(y_test,y_test_pred))

RMSE: 188701.63
R-sq of predictions:  -0.43556775080078447


In [ ]:
# from sklearn.svm import LinearSVR
# from sklearn.preprocessing import StandardScaler
# sc_X = StandardScaler()
# sc_y = StandardScaler()
# X_scale = sc_X.fit_transform(X)
# y_scale = sc_y.fit_transform(y)
# svmr = LinearSVR(epsilon=0, C=100, max_iter=1000, random_state=0).fit(X_scale, y_scale)
# print("Intercept: ", svmr.intercept_)
# print("Coefficient: ", svmr.coef_)
# print("R-sq of model: ", svmr.score(X_scale, y_scale))

In [ ]:
from sklearn.linear_model import Ridge
alpha_li = [0.5,2,5,100,1000,100000]
for a in alpha_li:
    rr = Ridge(alpha=a)
    rr.fit(X_poly_train, y_train)
    # print("Intercept: ", lr.intercept_)
    # print("Coefficient: ", lr.coef_)
    # print("R-sq of model with alpha",a,": ", lr.score(X_poly, y))
    y_test_pred = rr.predict(X_poly_test)
    print("RMSE with alpha",a,": %.2f" % mean_squared_error(y_test,y_test_pred, squared=False))
    print("R-sq of predictions with alpha",a,": ", r2_score(y_test,y_test_pred))

RMSE with alpha 0.5 : 176383.22
R-sq of predictions with alpha 0.5 :  -0.2542582353618097
RMSE with alpha 2 : 176383.21
R-sq of predictions with alpha 2 :  -0.25425800167552537
RMSE with alpha 5 : 176383.22
R-sq of predictions with alpha 5 :  -0.25425812865601527
RMSE with alpha 100 : 176383.17
R-sq of predictions with alpha 100 :  -0.2542575020138069
RMSE with alpha 1000 : 176383.25
R-sq of predictions with alpha 1000 :  -0.2542585675848472
RMSE with alpha 100000 : 176383.30
R-sq of predictions with alpha 100000 :  -0.2542592616536179


### Validation

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.1, random_state=6)
lr = LinearRegression()
lr.fit(X_train, y_train)
y_test_pred = lr.predict(X_test)
print("RMSE: %.2f" % mean_squared_error(y_test,y_test_pred, squared=False))
print("R-sq of predictions: ", r2_score(y_test,y_test_pred))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.1, random_state=6)
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.fit_transform(X_test)
lr.fit(X_train_poly, y_train)
y_test_pred = lr.predict(X_test_poly)
print("RMSE: %.2f" % mean_squared_error(y_test,y_test_pred, squared=False))
print("R-sq of predictions: ", r2_score(y_test,y_test_pred))

In [ ]:
kf = KFold(n_splits=10)

mse2_list = []
score_list = []
for train_index, test_index in kf.split(X):
    X_train = X[train_index]
    X_test = X[test_index]

    y_train = y.iloc[train_index]
    y_test = y.iloc[test_index]

    lm=LinearRegression()
    lm.fit(X_train, y_train)

    y_test_pred = lm.predict(X_test)
    mse = mean_squared_error(y_test, y_test_pred, squared=False)
    score = r2_score(y_test,y_test_pred)
    mse2_list.append(mse)
    score_list.append(score)
print("RMSE: %.2f" % np.mean(mse2_list))
print(mse2_list)
print(score_list)